# Deliverable 3. Create a Travel Itinerary Map.
----
1. Import the vacation CSV file. 
2. From the map above with pick 4 cities that are in close proximity (on the same continent) that the customer would travel to and create a directions layer map.
3. For the "travel_mode" use either "DRIVING", "BICYCLING", or "WALKING".
4. Take a screenshot of the route and save it. 
5. Then create a marker layer for the four cities. Each city will should have a pop-up marker that contains
    - The hotel name
    - The city
    - The country
    - The current weather description and the maximum temperature
---

## Make sure the initial dependencies and the Google API key are imported.

In [67]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets as gds

# Import API key
from config import google_api

In [21]:
# Configure gmaps
gmaps.configure(api_key=google_api)

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [22]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv('/Users/jeremy/Docs/OSU_Bootcamp/Mod_6_WeatherPy/World_Weather_Analysis/Vacation_Search/WeatherPy_Vacation.csv',index_col='City_ID')

# Display sample data
vacation_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
1,Georgetown,MY,84.13,scattered clouds,5.4112,100.3354,Cititel Penang
2,Isangel,VU,78.91,scattered clouds,-19.5500,169.2667,Tanna Lodge
5,Luanda,AO,77.00,overcast clouds,-8.8368,13.2343,EPIC SANA Luanda
7,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
15,Butaritari,KI,81.82,overcast clouds,3.0707,172.7902,Isles Sunset Lodge


## In this step, you will set-up the pop-up markers. Review the code to create a marker layer map of the vacation search results. This code is the same as in Deliverable 2.

In [23]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [24]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

## From the vacation search map, *choose four cities* that a customer might want to visit. They should be close together and in the same country.

In [36]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# The starting and ending city should be the same city.
vacation_start = vacation_df.loc[vacation_df['City']=='Ambanja']
vacation_end = vacation_df.loc[vacation_df['City']=='Ambanja']
vacation_stop1 = vacation_df.loc[vacation_df['City']=='Marovoay']
vacation_stop2 = vacation_df.loc[vacation_df['City']=='Beloha']
vacation_stop3 = vacation_df.loc[vacation_df['City']=='Ambovombe']

## Use the `to_numpy()` function and list indexing to write code to retrieve the latitude-longitude pairs as tuples from each city DataFrame.

In [66]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = tuple(vacation_start[['Lat','Lng']].to_numpy()[0])
end = tuple(vacation_end[['Lat','Lng']].to_numpy()[0])
stop1 = tuple(vacation_stop1[['Lat','Lng']].to_numpy()[0])
stop2 = tuple(vacation_stop2[['Lat','Lng']].to_numpy()[0])
stop3 = tuple(vacation_stop3[['Lat','Lng']].to_numpy()[0])

## Use the [gmaps documentation](https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer) to create a directions layer map using the variables from the previous step. Where the starting and ending city are the same city, the `waypoints` are the three other cities, and the `travel_mode` is either `"DRIVING"`, `"BICYCLING"`, or `"WALKING"`.

In [69]:
# Define a new figure object
fig = gmaps.figure()

# Create a direction layer map using the start and end latitude-longitude pairs, and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
vacation_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1,stop2,stop3],
        travel_mode='DRIVING')

# Add the layer to the map
fig.add_layer(vacation_itinerary)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

## Use the provided `concat()` function code snippet to combine the four separate city DataFrames into one DataFrame.

In [72]:
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)

# Display sample data
itinerary_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Ambanja,MG,82.33,clear sky,-13.6833,48.4500,Hotel Salama and Fast Food
1,Marovoay,MG,86.94,clear sky,-16.1000,46.6333,Hôtel les Oliviers Marovoay
2,Beloha,MG,87.44,clear sky,-25.1667,45.0500,NaN
3,Ambovombe,MG,79.90,clear sky,-25.1667,46.0833,RELAHARA


## Refactor the code from Step 6 to create a new marker layer map of the cities on the travel route.

In [79]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""
locations = []
# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]
# Get the latitude and longitude from each row and store in a new DataFrame.
locations =  itinerary_df[['Lat','Lng']]


,Lat,Lng
0,-13.6833,48.4500
1,-16.1000,46.6333
2,-25.1667,45.0500
3,-25.1667,46.0833


In [89]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

figure_layout = {
    'width': '400px',
    'height': '10000px',
    'border': '1px solid black',
    'padding': '1px'}
# Define a new figure object
fig = gmaps.figure(figure_layout)

# Add the layer to the map
fig.add_layer(marker_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))